In [1]:
import pyspark
from pyspark.sql import SQLContext
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
import pandas as pd
from pyspark.sql import SparkSession
from snorkel.labeling import PandasLFApplier,LFAnalysis
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()
pd_dev = pd.read_csv("review_dev_labelled.csv",header=0, index_col=0)
df_dev = spark.createDataFrame(pd_dev)

In [2]:
#Try to apply Labeling function
import pyspark.sql.functions as F
from snorkel.labeling import LabelModel
from snorkel.labeling.apply.spark import SparkLFApplier
import pandas as pd
import numpy as np
#from snorkel.labeling import ,LFAnalysis
from snorkel.labeling import LFAnalysis
from pyspark.sql import Row
from snorkel.labeling.lf import labeling_function
#from snorkel.labeling.lf.nlp_spark import spark_nlp_labeling_function
from snorkel.preprocess import preprocessor

ABSTAIN = -1
NEGATIVE = 0
POSITIVE = 1

In [5]:
pip install -U textblob

  Using cached textblob-0.15.3-py2.py3-none-any.whl (636 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
from textblob import TextBlob

@preprocessor(memoize=True)
def textblob_sentiment(x):
    scores = TextBlob(x.text)
    x.polarity = scores.sentiment.polarity
    x.subjectivity = scores.sentiment.subjectivity
    return x

In [7]:
#Labeliing Functions tried
@labeling_function(pre=[textblob_sentiment])
def textblob_polarity(x):
    return POSITIVE if x.polarity > 0.9 else ABSTAIN

In [8]:
@labeling_function(pre=[textblob_sentiment])
def textblob_subjectivity(x):
    return POISITIVE if x.subjectivity >= 0.5 else ABSTAIN

In [9]:
from snorkel.labeling import LabelingFunction

def keyword_lookup(x, keywords, label):
    if any(word in x.text.lower() for word in keywords):
        return label
    return ABSTAIN

In [10]:
def make_keyword_lf(keywords, label=0):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label)
    )

In [11]:
LabelingFunction(keyword_lookup, df_dev)

LabelingFunction <function keyword_lookup at 0x7f69ca0419d8>, Preprocessors: []

In [12]:
@labeling_function()
def high_useful(x):
    return POSITIVE if x.useful > 10 else ABSTAIN

In [13]:
@labeling_function()
def low_useful(x):
    return NEGATIVE if x.useful < 2 else ABSTAIN

In [14]:
@labeling_function()
def keyword_sour(x):
    return NEGATIVE if re.search("sour", x.text, flags=re.I) else ABSTAIN

In [15]:
@labeling_function()
def funny(x):
    return NEGATIVE if x.funny == 0 else ABSTAIN

In [16]:
@labeling_function()
def cool(x):
    return NEGATIVE if x.funny ==0 else ABSTAIN

In [17]:
@labeling_function()
def keyword_less(x):
    return POSITIVE if re.search("less", x.text, flags=re.I) else ABSTAIN

In [18]:
@labeling_function()
def keyword_misunderstanding(x):
    return POSITIVE if re.search("misunderstanding", x.text, flags=re.I) else ABSTAIN

In [19]:
@labeling_function()
def keyword_hells(x):
    return POSITIVE if re.search("hells", x.text, flags=re.I) else ABSTAIN

In [20]:
@labeling_function()
def short_comment(x):
    """Negative comments are often short, such as 'foods were insane'"""
    return NEGATIVE if len(x.text.split()) < 20 else ABSTAIN

In [21]:
@labeling_function()
def keyword_coupon(x):
    return POSITIVE if re.search("coupon", x.text, flags=re.I) else ABSTAIN

In [22]:
@labeling_function()
def keyword_love(x):
    return NEGATIVE if re.search("love", x.text, flags=re.I) else ABSTAIN

In [23]:
@labeling_function()
def keyword_great(x):
    return NEGATIVE if re.search("great", x.text, flags=re.I) else ABSTAIN

In [24]:
@labeling_function()
def keyword_rude(x):
    return POSITIVE if re.search("rude", x.text, flags=re.I) else ABSTAIN

In [25]:
@labeling_function()
def keyword_disappointment(x):
    return POSITIVE if re.search("disappointment", x.text, flags=re.I) else ABSTAIN

In [26]:
@labeling_function()
def keyword_table(x):
    return POSITIVE if re.search("table", x.text, flags=re.I) else ABSTAIN

In [27]:
@labeling_function()
def keyword_horrible(x):
    return POSITIVE if re.search("horrible", x.text, flags=re.I) else ABSTAIN

In [28]:
lis_1=["Rude","disappointment", "avoid", "overcooked", "pricey", "worst", "awkward" ,
       "awkwardly" , "expensive", "annoying", "tasteless", "bland", "Gross", "ewww",
       "slow","mess", "rubbery", "needed", "undercoocked", "wrong", "horrific",
       "grease", "horrible","overpriced", "overwhelmed", "dry" ,"tiny",
       "disgusting", "cardboard", "inattentive", "unfortunately",  "chewy" ,
       "unpleasant", "sucked", "displeased", "delayed", "loud", "crappy", "alright"]
list_2=["perfect", "always", "delicious", "enjoy", "amazing","great", "love",
        "excellent", "favourite" , "clean", "impeccable", "fabulous", "fav",
        "generous", "regulars", "outstanding", "regular", "bomb", "authentic",
        "reasonably","gem","amazing","excellent"]

In [29]:
@labeling_function()
def low_star(x):
    return POSITIVE if x.stars<4 else ABSTAIN

In [30]:
@labeling_function()
def keyword_connection(x):
    return POSITIVE if re.search("connection", x.text, flags=re.I) else ABSTAIN

In [31]:
@labeling_function()
def keyword_wifi(x):
    return NEGATIVE if re.search("wifi", x.text, flags=re.I) else ABSTAIN

In [32]:
@labeling_function()
def regex_wifi_connection(x):
    return POSITIVE if re.search(r"wifi.*connection", x.text, flags=re.I) else ABSTAIN

In [33]:
import re
lfs = [short_comment,keyword_love,keyword_horrible,keyword_rude,high_useful,keyword_sour,low_star]
applier = SparkLFApplier(lfs)
L_dev = applier.apply(df_dev.rdd)
g_label =np.array(df_dev.select('label').collect())
LFAnalysis(L_dev, lfs).lf_summary(g_label)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
short_comment,0,[0],0.048,0.022,0.010,17,7,0.708333
keyword_love,1,[0],0.194,0.046,0.034,72,25,0.742268
keyword_horrible,2,[1],0.010,0.008,0.002,4,1,0.800000
keyword_rude,3,[1],0.032,0.028,0.004,14,2,0.875000
high_useful,4,[1],0.010,0.008,0.004,4,1,0.800000
keyword_sour,5,[0],0.014,0.006,0.006,5,2,0.714286
low_star,6,[1],0.356,0.080,0.048,160,18,0.898876


In [36]:
lfs = [keyword_wifi,funny,keyword_table,regex_wifi_connection,keyword_great,low_useful,keyword_hells,keyword_misunderstanding,keyword_disappointment,keyword_less]
applier = SparkLFApplier(lfs)
L_dev = applier.apply(df_dev.rdd)
g_label =np.array(df_dev.select('label').collect())
LFAnalysis(L_dev, lfs).lf_summary(g_label)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
keyword_wifi,0,[0],0.006,0.006,0.006,2,1,0.666667
funny,1,[0],0.802,0.754,0.164,241,160,0.600998
keyword_table,2,[1],0.152,0.144,0.142,48,28,0.631579
regex_wifi_connection,3,[1],0.002,0.002,0.002,1,0,1.000000
keyword_great,4,[0],0.342,0.316,0.090,109,62,0.637427
low_useful,5,[0],0.772,0.722,0.150,232,154,0.601036
keyword_hells,6,[1],0.004,0.004,0.004,2,0,1.000000
keyword_misunderstanding,7,[1],0.002,0.002,0.002,1,0,1.000000
keyword_disappointment,8,[1],0.012,0.012,0.012,5,1,0.833333
keyword_less,9,[1],0.084,0.080,0.078,29,13,0.690476


In [41]:
label_model = LabelModel(cardinality=2)
label_model.fit(L_dev)
y_prob = label_model.predict_proba(L_dev)[:, 1] 
y_prob_sql_array = F.array([F.lit(y) for y in y_prob])
data_labeled = df_dev.withColumn("y_prob", y_prob_sql_array)

In [40]:
from snorkel.analysis import get_label_buckets

buckets = get_label_buckets(L_dev[:, 0], L_dev[:, 1])
pd_dev.iloc[buckets[(ABSTAIN, NEGATIVE)]].sample(10, random_state=1)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,index_1,index_2,label
#,,,,,,,,,,,,
215,hdnrqrYJE1RkoUH4MFq3Ag,1,09/11/2011 19:55,0,po13Pb4pZcSso6ilDMy3wA,2,Part of me can't believe that this place has g...,1,GPASJQMtRKQ5kxlXK29UjQ,1.762400e+04,215,1
453,ViErpcikhbAjsDcpdAfwSQ,0,11/11/2010 18:03,0,9tv8hCom-dPAQDC6PQXIRQ,5,HOLY CRAP this is AMAZING!!!!! I love food an...,0,oYwE3G7SDh5r3K6vQ_IJwQ,3.690000e+11,453,0
313,7eQoxWr9RzyCB9IFvqHUPA,0,10/06/2018 04:46,0,U75e6Ji3WXoc8ReZHAWuvw,1,Horrible experience never order delivery from ...,0,jojhw3HqtaVrHPNJV607YQ,3.690000e+11,313,1
340,g8OnV26ywJlZpezdBnOWUQ,0,05/11/2013 18:17,0,JDJBYluFfSNamEXayi2ibQ,1,I don't like to due damage to anyone's busines...,0,FdH4OV7ofT243yuJYlQTHA,7.390000e+11,340,1
355,5k8kyvvReTf1RMq4jpayQg,0,11/08/2017 05:43,0,dqC9nWK-5NCe61CgmzOW0w,5,This place has the best customer service and h...,0,PR5wNJfKhuSUVrco1Fkuww,2.576981e+10,355,0
290,L2p0vO3fsS2LC6hhQo3CzA,0,23/05/2015 17:27,0,TPfLAOHghKphm0UuFeYDGQ,5,I had a cheese omelette with spinach and onion...,0,220gLMOenqRJ_RCc_7c6JQ,1.130000e+12,290,0
146,2zx0Pt5OeHhjAZoNRqYTog,0,09/07/2018 18:37,0,CLf3hWQ43s0sUPBg6cP-Mw,1,"Relatives coming into town, wanted everyone to...",2,HRNysSGlkeq-Fjud0k0Y4Q,4.470000e+11,146,1
439,r5pBbEFHWxlBBHs3N8QiPA,0,20/09/2014 19:48,0,onQcys9WC6tt5kn36XTfeg,1,I've given this place so many chances with the...,2,n7XFMUCNkvoQmRUI5lXzfg,1.310000e+12,439,1
237,65eDIQ8bpMJFtgdyX7ulVA,0,01/06/2017 19:01,0,kRYVOKTKeXvsUOr-4nvhVA,4,Im back again! The food is still worth the U T...,1,RAm1-WTEAt7AowUz9vdVZQ,7.390000e+11,237,0


In [42]:
buckets = get_label_buckets(L_dev[:, 0], L_dev[:, 1])
pd_dev.iloc[buckets[(NEGATIVE,ABSTAIN)]].sample(10, random_state=1)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,index_1,index_2,label
#,,,,,,,,,,,,
275,e7bcP2P8WO0NFe1DjjoYQA,0,12/02/2012 07:18,0,OvWKN5BPB2sni7q9c5VZGA,3,Fried chicken is really good...great for groups!,1,J9c0XM-wV_KQmtZYxvIFFQ,1.650000e+12,275,0
106,gkIyPrNGpF4EGqoYWe2KBQ,0,22/07/2016 00:13,0,SWuGaeQP6kSkz9O0RkDX4g,2,I ordered the chicken pecan salad disappointin...,0,DGPCuDVRDkF6vJi9H1PE1A,1.680000e+12,106,0
410,medCzwjjGeReqxOt-iPVTg,0,20/12/2016 21:57,0,vOZJHBwhx8EunZCh-sGP8g,1,They have a terrible service it's really bad d...,0,aomktGLkePzI5Bv3Rg3KJQ,1.580000e+12,410,1
79,XFXkImvmUFUERCuZeiuZtw,0,14/10/2018 20:54,0,zIyb6X2aSSiJRClyLuY1nQ,5,Bomb blackened Haddock Tacos! Definitely worth...,0,MS-zI-hENEEjO6Tw9Z43LA,6.870000e+11,79,0
417,xqMzRiPK2SkN9upGukPzYw,1,20/05/2014 00:06,2,6TkWO6b0hSeAZ8Xja1-bHw,4,The mayo packets are warm at this establishmen...,2,vpFpiBxxXfGC85Nf-kNPng,7.990000e+11,417,1
289,oN8HDF_cPDaeFB69yC6I5A,0,02/09/2016 05:46,0,fEt2Pk3n02DzOfUki2gC4w,5,Just go! You won't be disappointed!\nFriendly ...,0,S6WAo9h7PHqUDbog60pzjw,1.020000e+12,289,0
428,gc78dwGe4EP6resjJHJEbA,0,23/01/2017 03:30,0,1aYZSjIxAQSgsiKbFWWKNQ,5,"food is always good and the service is fine, t...",0,sJDMHqOyTlO3bDKPxbRiWg,6.960000e+11,428,0
243,xU_QqrlaxXlf2SKoNIaGpg,0,09/09/2014 01:37,1,p6FUIkP4wVYhIhnac9KDrQ,5,Had the chicken Slovakia it was very good. Ser...,1,PsFkWVr9Ql3BRtqipOkssw,1.310000e+12,243,0
41,AUOlDSQ8R2wIkN4lQ4kVDg,0,20/10/2013 21:54,0,olNkuRtYu_dRsYzdxaSQOQ,5,Food is great however the service can definite...,0,sM_oORsDKeFuPRHWlbK6nw,1.070000e+12,41,1


In [41]:
lfs = [short_comment,keyword_love,keyword_horrible,keyword_rude,high_useful,keyword_sour,low_star,keyword_wifi,funny,keyword_table,regex_wifi_connection,keyword_great,low_useful,keyword_hells,keyword_misunderstanding,keyword_disappointment,keyword_less]
applier = SparkLFApplier(lfs)
L_dev = applier.apply(df_dev.rdd)
g_label =np.array(df_dev.select('label').collect())
LFAnalysis(L_dev, lfs).lf_summary(g_label)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
short_comment,0,[0],0.048,0.046,0.010,17,7,0.708333
keyword_love,1,[0],0.194,0.188,0.068,72,25,0.742268
keyword_horrible,2,[1],0.010,0.010,0.010,4,1,0.800000
keyword_rude,3,[1],0.032,0.032,0.030,14,2,0.875000
high_useful,4,[1],0.010,0.010,0.008,4,1,0.800000
keyword_sour,5,[0],0.014,0.014,0.008,5,2,0.714286
low_star,6,[1],0.356,0.336,0.328,160,18,0.898876
keyword_wifi,7,[0],0.006,0.006,0.006,2,1,0.666667
funny,8,[0],0.802,0.780,0.342,241,160,0.600998
keyword_table,9,[1],0.152,0.148,0.142,48,28,0.631579
